In [1]:
from bins import Bins
from utils import calcAllChi2, calcOneChi2, HistMaker
from ROOT import TFile, TH1
import math

bins = Bins.readFrom("ranges.yml")
histMakerData = HistMaker("apr12_diele_088_090_ag123ag_2500A_accepted_np_2.dat", "_data", bins)
histsData = histMakerData.makeHists()
histMakerMC = HistMaker("medium_isotropic_eff_ag1230ag_np_9deg.dat", "_MC", bins    )

#outfile = TFIle("out.root","RECREATE")
allHistsMC = []

#calcAllChi2(histsMC, histsData)

Welcome to JupyROOT 6.28/04
iter 0
Before processing events 16:10:23
After processing events 16:10:23
iter 1
Before processing events 16:10:58
After processing events 16:11:00


In [2]:
import torch

torch.manual_seed(0)

import gpytorch
import botorch

import matplotlib.pyplot as plt

plt.style.use("bmh")
plt.rcParams["figure.figsize"] = (8, 6)

from tqdm.notebook import tqdm

import warnings

In [3]:
TH1.SetDefaultSumw2

N_PARAMS = 3

grid_x = torch.linspace(-1,1,101)

grid_x1, grid_x2, grid_x3 = torch.meshgrid(grid_x, grid_x, grid_x, indexing="ij")

xs = torch.vstack([grid_x1.flatten(), grid_x2.flatten(), grid_x3.flatten()]).transpose(-2,-1)

In [4]:
#lb = -1
#ub = 1

#bounds = torch.tensor([[lb]*N_PARAMS, [ub]*N_PARAMS], dtype=torch.float)
bounds = torch.tensor([[0, 0,        -1],
                       [1, 2*math.pi, 1]], dtype=torch.float)
bounds

tensor([[ 0.0000,  0.0000, -1.0000],
        [ 1.0000,  6.2832,  1.0000]])

In [5]:
class GPModel(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    _num_outputs = 1

    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=1)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def fit_gp_model(train_x, train_y, num_train_iters=500):
    # declare the GP
    noise = 1e-4

    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(train_x, train_y, likelihood)
    model.likelihood.noise = noise

    # train the hyperparameter (the constant)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    for i in range(num_train_iters):
        optimizer.zero_grad()

        output = model(train_x)
        loss = -mll(output, train_y)

        loss.backward()
        optimizer.step()

    model.eval()
    likelihood.eval()

    return model, likelihood

In [6]:
num_queries = 200
num_repeats = 1
num_samples = 1

In [7]:
def lambdas(xx):
   # return x[0], x[1], x[2]
    r, phi, z = xx[0], xx[1], xx[2]
    x = r*math.cos(phi)
    y = r*math.sin(phi)
    lambda_theta = 0.5 * (2*x + z)
    lambda_phi   = 0.25 * (-2 - 2*x + 3*z)
    lambda_theta_phi = y/math.sqrt(2.)
    return lambda_theta, lambda_phi, lambda_theta_phi

def constraint1(xx):
    print("In the constraint fucntion, xx = ", xx)
    input_shape = xx.shape
    ll = list(xx.shape)
    ll[-1] = 1
    if len(input_shape)>1:
        xx = torch.reshape(xx, input_shape[1:])
        output_shape = tuple(ll)
        result = 1+xx[0,0]+xx[0,1]
    else:
        result = 1+xx[0]+xx[1]
        output_shape = (1,)
        
    print(xx)
    print(input_shape)
    print(result)
    print(output_shape)
    
    return torch.reshape(torch.tensor([result],requires_grad=xx.requires_grad),output_shape)

#def cost(xx):
#    def generator(xx):
#        for x in xx:
#            lambda_theta, lambda_phi, lambda_theta_phi = lambdas(x)#
#
#            result = 0
#
#            if pow(1-lambda_phi, 2) - pow(lambda_theta-lambda_phi, 2) < 4*pow(lambda_theta_phi, 2):
#                result = 1
#            if 1 + lambda_theta + 2*lambda_phi < 0:
#                result = 1

#            yield torch.tensor([result])
#    return torch.stack([a for a in generator(xx)])
    
def objective(xx):
    def generator(xx):
        for x in xx:
            lambda_theta, lambda_phi, lambda_theta_phi = lambdas(x)

            histsMC = histMakerMC.makeHists(lambda_theta, lambda_phi, lambda_theta_phi)
            chi2, ndf = calcOneChi2(histsMC[0][0], histsData[0][0])
            allHistsMC.append(histsMC[0][0])
            if not chi2 or not ndf:
                return torch.tensor([0])
            yield torch.tensor([(chi2 / ndf)])
           # yield torch.tensor([1./(chi2 / ndf)])
    return torch.stack([a for a in generator(xx)])

In [8]:
# -2 is the default value when no feasible has been found
default_value = -2
feasible_incumbents = torch.ones((num_repeats, num_queries)) * default_value

for trial in range(num_repeats):
    print("trial", trial)

    torch.manual_seed(trial)
    train_x = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(num_samples, 3)
    train_utility = objective(train_x)

    for i in tqdm(range(num_queries)):
        print("query", i)

        feasible_incumbents[trial, i] = train_utility.max()
        utility_model, utility_likelihood = fit_gp_model(
            train_x, train_utility.squeeze(-1)
        )
        best_f = train_utility.max()
                
        policy = botorch.acquisition.monte_carlo.qExpectedImprovement(
            model=utility_model,
            best_f=train_utility.max(),
        )

        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=RuntimeWarning)
            next_x, acq_val = botorch.optim.optimize_acqf(
                policy,
                bounds=bounds,
                q=1,
                num_restarts=40,
                raw_samples=100,
            #    nonlinear_inequality_constraints=[constraint1],
                inequality_constraints=[(torch.tensor([0,2]),torch.tensor([-1.0,-0.5],dtype=torch.float),-0.5)],
            #    ic_generator=botorch.optim.initializers.gen_batch_initial_conditions
            )

        next_utility = objective(next_x)

        train_x = torch.cat([train_x, next_x])
        train_utility = torch.cat([train_utility, next_utility])


trial 0


  0%|          | 0/200 [00:00<?, ?it/s]

query 0
query 1
query 2
query 3
query 4
query 5
query 6
query 7
iter 10
Before reweighting hists 16:11:15
After reweighting hists 16:11:15
query 8
query 9
query 10
query 11
query 12
query 13
query 14
query 15
query 16
query 17
iter 20
Before reweighting hists 16:11:28
After reweighting hists 16:11:28
query 18
query 19
query 20
query 21
query 22
query 23
query 24
query 25
query 26
query 27
iter 30
Before reweighting hists 16:11:49
After reweighting hists 16:11:49
query 28
query 29
query 30


/Users/harabasz/miniforge3/envs/pyroot/lib/python3.11/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


query 31
query 32


...

/Users/harabasz/miniforge3/envs/pyroot/lib/python3.11/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


iter 200
Before reweighting hists 16:24:02
After reweighting hists 16:24:02
query 198


/Users/harabasz/miniforge3/envs/pyroot/lib/python3.11/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


query 199


/Users/harabasz/miniforge3/envs/pyroot/lib/python3.11/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


In [9]:
torch.save(feasible_incumbents, f"./incumbents.pth")
fout = TFile("out.root", "RECREATE")
fout.cd()
for hist in allHistsMC:
    print ("Writing hist: ", hist.GetName())
    hist.Write()
for j, hists in enumerate(histsData):
    for k, hist in enumerate(hists):
            hist.Write()
            print ("Writing hist: ", j, k, hist)
fout.Close()

Writing hist:  hist_0to150_m06m02_MC_iter2
Writing hist:  hist_0to150_m06m02_MC_iter3
Writing hist:  hist_0to150_m06m02_MC_iter4
Writing hist:  hist_0to150_m06m02_MC_iter5
Writing hist:  hist_0to150_m06m02_MC_iter6
Writing hist:  hist_0to150_m06m02_MC_iter7
Writing hist:  hist_0to150_m06m02_MC_iter8
Writing hist:  hist_0to150_m06m02_MC_iter9
Writing hist:  hist_0to150_m06m02_MC_iter10
Writing hist:  hist_0to150_m06m02_MC_iter11
Writing hist:  hist_0to150_m06m02_MC_iter12
Writing hist:  hist_0to150_m06m02_MC_iter13
Writing hist:  hist_0to150_m06m02_MC_iter14
Writing hist:  hist_0to150_m06m02_MC_iter15
Writing hist:  hist_0to150_m06m02_MC_iter16
Writing hist:  hist_0to150_m06m02_MC_iter17
Writing hist:  hist_0to150_m06m02_MC_iter18
Writing hist:  hist_0to150_m06m02_MC_iter19
Writing hist:  hist_0to150_m06m02_MC_iter20
Writing hist:  hist_0to150_m06m02_MC_iter21
Writing hist:  hist_0to150_m06m02_MC_iter22
Writing hist:  hist_0to150_m06m02_MC_iter23
Writing hist:  hist_0to150_m06m02_MC_ite

In [10]:
t = torch.tensor( [ [-1.0000, -0.4678,  0.5433]])
print(t)
print(t.shape)

tensor([[-1.0000, -0.4678,  0.5433]])
torch.Size([1, 3])


In [11]:
from ipywidgets import interact, Layout, IntSlider
import numpy as np

def oneplot(ax, tensor, index, cmap):
        tensor_3d = torch.reshape(tensor, (101,101,101))
        pos = ax.imshow(tensor_3d[index], cmap=cmap, interpolation="nearest", origin="lower", 
        vmin=tensor.min(), vmax=tensor.max(), extent=[-1,1,-1,1])
        plt.colorbar(pos)

with torch.no_grad():
    predictive_distribution = utility_likelihood(utility_model(xs))
    predictive_mean = predictive_distribution.mean
    predictive_lower, predictive_upper = predictive_distribution.confidence_region()

def f(x):
        cmap = "gist_rainbow"
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(6, 6))
        oneplot(ax[0][0], predictive_mean, x, cmap)
        oneplot(ax[0][1], predictive_lower, x, cmap)
        oneplot(ax[1][1], predictive_upper, x, cmap)
        oneplot(ax[1][0], predictive_upper-predictive_lower, x, cmap)

interact(f, x=IntSlider(50, 0, 100, 1, layout=Layout(width='500px')))


interactive(children=(IntSlider(value=50, description='x', layout=Layout(width='500px')), Output()), _dom_clas…

<function __main__.f(x)>

In [12]:
t = torch.tensor([[[0.0538, 0.2096, 0.2256]]])
t = torch.reshape(t,(1,3))
(1,2,3)[1:]

(2, 3)

In [13]:
(feasible_incumbents==feasible_incumbents.max()).nonzero()

tensor([[  0,  94],
        [  0,  95],
        [  0,  96],
        [  0,  97],
        [  0,  98],
        [  0,  99],
        [  0, 100],
        [  0, 101],
        [  0, 102],
        [  0, 103],
        [  0, 104],
        [  0, 105],
        [  0, 106],
        [  0, 107],
        [  0, 108],
        [  0, 109],
        [  0, 110],
        [  0, 111],
        [  0, 112],
        [  0, 113],
        [  0, 114],
        [  0, 115],
        [  0, 116],
        [  0, 117],
        [  0, 118],
        [  0, 119],
        [  0, 120],
        [  0, 121],
        [  0, 122],
        [  0, 123],
        [  0, 124],
        [  0, 125],
        [  0, 126],
        [  0, 127],
        [  0, 128],
        [  0, 129],
        [  0, 130],
        [  0, 131],
        [  0, 132],
        [  0, 133],
        [  0, 134],
        [  0, 135],
        [  0, 136],
        [  0, 137],
        [  0, 138],
        [  0, 139],
        [  0, 140],
        [  0, 141],
        [  0, 142],
        [  0, 143],


In [25]:
#train_x[94]
lambdas(train_x[94])
#1./math.sqrt(2)

(tensor(1.6643), tensor(1.6638), tensor(0.0003))